In [13]:
import keras
import numpy as np
import tensorflow
import matplotlib.pyplot as plt
import segmentation_models as sm

### Import Data

In [14]:
from Modelling.Baselines.Data_Preprocessing import import_labeled_data,import_labeled_photos
X,Y=import_labeled_data()

In [15]:
from numpy.random.mtrand import random

def Split_Train_Test_Photos(X,Y):
    frac = 0.2  # how much of a/b do you want to exclude
    # generate a list of indices to exclude. Turn in into a set for O(1) lookup time
    #inds_x = set(random.sample(list(range(len(X))), int(frac*len(X))))
    #inds_y = set(random.sample(list(range(len(X))), int(frac*len(X))))
    
    # use `enumerate` to get list indices as well as elements. 
    # Filter by index, but take only the elements
    #x_val = [n for i,n in enumerate(X) if i not in inds_x]
    #x_train = [n for i,n in enumerate(X) if i in inds_x]
    #y_val = [n for i,n in enumerate(Y) if i not in inds_y]
    #y_train = [n for i,n in enumerate(Y) if i in inds_y]
    x_val = X[1]
    x_train = X[0]
    y_val = Y[1]
    y_train = Y[0]

    return x_train,x_val,y_train,y_val

In [16]:


BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

# load your data
from sklearn.model_selection import train_test_split
X, Y= import_labeled_photos()
x_train,x_val,y_train,y_val=Split_Train_Test_Photos(X,Y)

# preprocess input
x_train = preprocess_input(x_train)
x_val = preprocess_input(x_val)

from segmentation_models import Unet

# read/scale/preprocess data


# define number of channels
N = 110

# define model
model = Unet(backbone_name='resnet34', encoder_weights=None, input_shape=(None, None, N))

model.compile(
    'Adam',
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score],
)

# fit model
# if you use data generator use model.fit_generator(...) instead of model.fit(...)
# more about `fit_generator` here: https://keras.io/models/sequential/#fit_generator


In [17]:
history=model.fit(
   x=x_train,
   y=y_train,
   epochs=10,
   validation_data=(x_val, y_val),
)

Epoch 1/10


ValueError: in user code:

    File "C:\Users\Timo\PycharmProjects\pds_gyrocopter\env\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Timo\PycharmProjects\pds_gyrocopter\env\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Timo\PycharmProjects\pds_gyrocopter\env\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Timo\PycharmProjects\pds_gyrocopter\env\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Timo\PycharmProjects\pds_gyrocopter\env\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Timo\PycharmProjects\pds_gyrocopter

In [ ]:
# Plot training & validation iou_score values
plt.figure(figsize=(30, 5))
plt.subplot(121)
plt.plot(history.history['iou_score'])
plt.plot(history.history['val_iou_score'])
plt.title('Model iou_score')
plt.ylabel('iou_score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(122)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
from segmentation_models import Unet

